# Poem Generator

In [1]:
from re import sub
from typing import List

def load_text(filename:str)->List[str]:
    '''
        Reads the .txt file.
        
        Parameter
        ---------
        `filename`: str
            The name of the poems file.
            
        Returns
        -------
        A list containing each strophe's content.
    '''
    with open(f'/kaggle/input/poe-vs-frost/{filename}', 'r') as f:
        strophe_delim = '\n\n'
        return sub('\n\u2009\n', strophe_delim, f.read()).split(strophe_delim)
    
txt_frost = load_text('05_robert_frost.txt')

## Text Treatment
* Text Normalization, \<EOS\> tag and turning new line a valid character.

In [2]:
import string
from re import sub
from nltk import word_tokenize

def remove_punctuation(s:str)->str:
    '''
        Removes punctuation from a string.
        
        Parameter
        ---------
        s: `str`
            The provided string.
        
        Returns
        -------
        The treated string.
    '''
        
    translation_table = str.maketrans('', '', string.punctuation)
    return s.lower().strip().translate(translation_table)

def eos(s:str)->str:
    '''
        Creates an End-Of-Sentence tag at the end of the string.

        Parameter
        ---------
        s: `str`
            The provided string.

        Returns
        -------
        The treated string.
    '''
    return s+' <eos>' if '<eos>' not in s else s

def new_line(s:str)->str:
    '''
        Turns \n character a token
        
        Parameter
        ---------
        s: `str`
            The provided string.


        Returns
        -------
        The treated string.
    '''
    return sub('\n', ' \n ', s)
    
def treat(s:str)->str:
    '''
        Applies all transformations mentioned above in a text.
        
        Parameter
        ---------
        s: `str`
            The provided string.

        Returns
        -------
        The treated string.
    '''
    s = remove_punctuation(s)
    s = eos(s)
    s = new_line(s)
    return s.split()

txt_frost = list(map(treat, txt_frost))

## Creating the Markov Model

In [3]:
import numpy as np
from collections import Counter
from typing import List
class MarkovModel:
    
    def __pi(self, X:List[str]):
        self.pi = Counter(x[0] for x in X)
        self.pi = {token:count/len(X) for token, count in self.pi.items()}
        
    def __a(self, X:List[str]):
        self.a = Counter(x[0]+'<sep>'+x[1] for x in X)
        self.a = {key.split('<sep>')[0]:{key.split('<sep>')[1]:self.a[key]} for key in self.a.keys()}
        
        for key in self.a.keys():
            self.a[key] = {j:v/len(X) for j,v in self.a[key].items()}
            
    def __a2(self, X:List[str]):
        self.a2 = Counter(x[i-2]+'<sep>'+x[i-1]+'<sep>'+x[0] for x in X for i in range(2, len(x)))
        self.a2 = {key.split('<sep>')[0]:{key.split('<sep>')[1]:{key.split('<sep>')[2]:self.a2[key]}}
                   for key in self.a2.keys()}
        
    def fit(self, X:List[str]):
        self.__pi(X)
        self.__a(X)
        self.__a2(X)
        
a = MarkovModel()
a.fit(txt_frost)
a.a2

{'two': {'oclock': {'a': 1}},
 'roads': {'diverged': {'i': 1}},
 'diverged': {'in': {'i': 1}},
 'in': {'ways': {'i': 1}},
 'a': {'double': {'a': 1}},
 'yellow': {'strands': {'once': 1}},
 'wood': {'and': {'i': 1}},
 'and': {'sport': {'a': 1}},
 'sorry': {'i': {'once': 1}},
 'i': {'suppose': {'i': 1}},
 'could': {'sell': {'i': 1}},
 'not': {'melt': {'a': 1}},
 'travel': {'both': {'two': 1}},
 'both': {'of': {'i': 1}},
 'be': {'expecting': {'i': 1}},
 'one': {'level': {'it': 2}},
 'traveler': {'long': {'two': 1}},
 'long': {'time': {'i': 1}},
 'stood': {'the': {'i': 1}},
 'looked': {'like': {'mother': 1}},
 'down': {'ryans': {'i': 1}},
 'as': {'winter': {'so': 1}},
 'far': {'before': {'i': 1}},
 'to': {'wake': {'a': 1}},
 'where': {'he': {'it': 1}},
 'it': {'worth': {'a': 1}},
 'bent': {'in': {'two': 1}},
 'the': {'years': {'it': 1}},
 'then': {'its': {'i': 1}},
 'took': {'the': {'mother': 1}},
 'other': {'way': {'i': 1}},
 'just': {'the': {'i': 1}},
 'fair': {'i': {'i': 1}},
 'having': 

<p style='color:red'> Revisar matriz A de segunda ordem e normalizar seus valores </p>